# Lab 3: Policy Search

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The player **taking the last object wins**.

* Task3.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task3.2: An agent using evolved rules
* Task3.3: An agent using minmax
* Task3.4: An agent using reinforcement learning

## Deadlines ([AoE](https://en.wikipedia.org/wiki/Anywhere_on_Earth))

* Sunday, December 4th for Task3.1 and Task3.2
* Sunday, December 11th for Task3.3 and Task3.4
* Sunday, December 18th for all reviews

In [38]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from functools import reduce
from operator import xor

logging.basicConfig(format="%(asctime)s %(levelname)s: %(message)s", level=logging.DEBUG)

## The *Nim* and *Nimply* classes

In [39]:
Nimply = namedtuple("Nimply", "row, num_objects")
Genome = namedtuple("Genome", "aggressivity, longest_first, how_many")
Individual = namedtuple("Individual", "genome, fitness")

In [40]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply     
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

## Task 3.1

### My fixed-rule agent

In [41]:
def make_strategy(genome: Genome) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        # data = cook_status(state)
        aggressive: bool = random.random() < genome.aggressivity
        longest_first: bool = random.random() < genome.longest_first
        how_many_coeff: float = genome.how_many

        # Sort the rows based on the number of elements, the sort is descending or ascending based on if longest_first or not
        row_indexes = sorted((i for i in range(len(state.rows)) if state.rows[i] > 0), key=lambda elem: state.rows[elem], reverse=longest_first)

        # Select randomly one of the first 50% rows
        selected_row_index = random.choice(row_indexes[:int(0.5*len(row_indexes))+1])

        # Decide to take at least 1 or half of the objects
        take_at_least = 0 if not aggressive else state.rows[selected_row_index]//2
        
        # Decide to take or not a part of the remaining objects
        take_n = max(1, min(take_at_least + int(state.rows[selected_row_index]//2*how_many_coeff), state.rows[selected_row_index]))
        ply = Nimply(selected_row_index, take_n)

        return ply

    return evolvable

# Task 3.1 agent
not_evovled = make_strategy(Genome(0.5, 0.5, 0.5))

### Other fixed-rule agents

In [42]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [43]:
def nim_sum(state: Nim) -> int:
    result = reduce(xor, state.rows)
    return result


def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    # cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    # cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    # cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

In [44]:
def optimal_strategy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

## Task 3.2

The agent base policy it's the same of task 3.1, but now I evolve its parameters 

In [45]:
NIM_SIZE = 10


def evaluate(strategy: Callable, opponent: Callable=optimal_strategy, NUM_MATCHES=100) -> float:
    strategies = (strategy, opponent)
    won = 0

    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = strategies[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won / NUM_MATCHES

## Evolve genome

In [46]:
def tournament(population, k=10):
    return max(random.choices(population, k=k), key=lambda x: x.fitness)

def make_offspring(p1: Individual, p2: Individual, current_best_strategy):
    new_genome = []
    for i in range(len(p1.genome)):
        gene = p1.genome[i] if random.random() > 0.5 else p2.genome[i] # inherit from p1 or p2 based on randomness
        gene += random.gauss(0, 0.25) # tweak
        gene = max(0, min(gene, 1)) # clip to avoid unammissible solutions
        new_genome.append(gene)

    new_genome = Genome(*new_genome)
    individual = Individual(new_genome, evaluate(make_strategy(new_genome), current_best_strategy)) # create individual and compute fitness
    
    return individual

In [47]:
ITERATIONS = 20
POPULATION_SIZE = 40
OFFSPRING_SIZE = 200
genomes = [Genome(0.5 + random.random()/10, 0.5 + random.random()/10, 0.5 + random.random()/10) for _ in range(POPULATION_SIZE)]

# the initial population fitness is computed against gabriele
population = list(map(lambda genome: Individual(genome, evaluate(make_strategy(genome), gabriele)), genomes))

for i in range(ITERATIONS):
    logging.debug(f"Starting iteration {i}. Current fitness: {population[0].fitness}")
    logging.debug(f"Current best:\nAggressivity\tLongest first\tHow many\n\
{population[0].genome.aggressivity:.2f}\t\t{population[0].genome.longest_first:.2f}\t\t{population[0].genome.how_many:.2f}")

    offspring = []
    for _ in range(OFFSPRING_SIZE):
        p1, p2 = tournament(population, k=1), tournament(population, k=1)
        o = make_offspring(p1, p2, make_strategy(population[0].genome))
        offspring.append(o)
    population.extend(offspring)
    population = sorted(population, key=lambda individual: individual.fitness, reverse=True)[:POPULATION_SIZE]

evolved_individual = population[0]

2022-12-04 13:54:31,771 DEBUG: Starting iteration 0. Current fitness: 0.08
2022-12-04 13:54:31,779 DEBUG: Current best:
Aggressivity	Longest first	How many
0.53		0.59		0.58
2022-12-04 13:54:39,433 DEBUG: Starting iteration 1. Current fitness: 0.67
2022-12-04 13:54:39,433 DEBUG: Current best:
Aggressivity	Longest first	How many
0.50		0.93		0.00
2022-12-04 13:54:45,244 DEBUG: Starting iteration 2. Current fitness: 0.67
2022-12-04 13:54:45,244 DEBUG: Current best:
Aggressivity	Longest first	How many
0.50		0.93		0.00
2022-12-04 13:54:51,043 DEBUG: Starting iteration 3. Current fitness: 0.67
2022-12-04 13:54:51,043 DEBUG: Current best:
Aggressivity	Longest first	How many
0.50		0.93		0.00
2022-12-04 13:54:56,690 DEBUG: Starting iteration 4. Current fitness: 0.71
2022-12-04 13:54:56,697 DEBUG: Current best:
Aggressivity	Longest first	How many
0.67		0.73		1.00
2022-12-04 13:54:59,746 DEBUG: Starting iteration 5. Current fitness: 0.71
2022-12-04 13:54:59,746 DEBUG: Current best:
Aggressivity	Lo

## Final match

In [48]:
strategies = (make_strategy(evolved_individual.genome), optimal_strategy)

nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategies[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

2022-12-04 13:55:41,856 DEBUG: status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
2022-12-04 13:55:41,856 DEBUG: status: After player 0 -> <1 3 5 7 9 11 13 1 17 19 21>
2022-12-04 13:55:41,856 DEBUG: status: After player 1 -> <1 3 5 7 9 11 13 1 8 19 21>
2022-12-04 13:55:41,864 DEBUG: status: After player 0 -> <1 3 5 7 9 11 13 1 8 19 1>
2022-12-04 13:55:41,864 DEBUG: status: After player 1 -> <1 3 5 7 9 11 13 1 8 7 1>
2022-12-04 13:55:41,864 DEBUG: status: After player 0 -> <1 3 5 7 9 11 1 1 8 7 1>
2022-12-04 13:55:41,869 DEBUG: status: After player 1 -> <1 3 5 7 5 11 1 1 8 7 1>
2022-12-04 13:55:41,869 DEBUG: status: After player 0 -> <1 3 5 1 5 11 1 1 8 7 1>
2022-12-04 13:55:41,869 DEBUG: status: After player 1 -> <1 3 3 1 5 11 1 1 8 7 1>
2022-12-04 13:55:41,869 DEBUG: status: After player 0 -> <1 3 1 1 5 11 1 1 8 7 1>
2022-12-04 13:55:41,872 DEBUG: status: After player 1 -> <1 1 1 1 5 11 1 1 8 7 1>
2022-12-04 13:55:41,872 DEBUG: status: After player 0 -> <0 1 1 1 5 11 1 1 8 7 1>
2